In [1]:
from lsst.daf.butler import Butler
from astropy.table import Table
import pylab
import numpy
from astropy.stats import mad_std
from tqdm import tqdm
from astropy.io import fits
import pandas as pd
from scipy import stats
from multiprocessing import Pool
import pickle
import os

In [2]:
repo_path = "embargo_new"
butler = Butler(repo_path,instrument='LSSTCam')
registry = butler.registry

We query what collections we have in the repository

In [3]:
#13590
#13535
#13525
# https://confluence.slac.stanford.edu/pages/viewpage.action?pageId=405094202
collections = list(registry.queryCollections("*stability*E1496*"))
collections

['u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35/20241025T082704Z',
 'u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35']

We query what dataset are in this collections

In [4]:
sensor="R23_S00"
detector=list(registry.queryDimensionRecords("detector",instrument='LSSTCam',where=f"detector.full_name='{sensor}'"))[0].id
detector

99

In [8]:
import traceback
def getcoef( detector ):
    print(collections,detector)
    df=butler.get('flat_gain_stability_stats',collections=collections,detector=detector)

    expids=set(list(df['exposure']))

    mjd=[]
    temps1=[]
    temps2=[]
    expid=[]
    for ref in tqdm(sorted( expids )[::]):
        try:
            header = butler.get('raw.metadata',
                                dataId={"exposure": ref},
                                collections=collections,detector=detector)
            expid.append(ref)
            temps1.append(header['TEMP6'])
            temps2.append(header['TEMP10'])
            mjd.append(header['MJD'])
        except:
            traceback.print_exc()
            pass

    dfmerged=pd.merge(df, pd.DataFrame({'exposure': expid, 'TEMP6': temps1, 'TEMP10': temps2, 'MJD': mjd }), on='exposure', how='inner')

    ret = {}
    for amp in list(set(dfmerged.amp_name)):
        subset=dfmerged[dfmerged['amp_name']==amp]
        y0=(subset['median']/subset['pd_integral']).min()
        x=(subset['TEMP6']+subset['TEMP10'])/2
        y=subset['median']/subset['pd_integral']/y0
        slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
        ret.update( {amp: { 
            "mjd": subset['MJD'],
            "x": x,
            "y": y,
            "y0": y0,
            "slope": slope,
            "intercept": intercept,
            "r_value": r_value,
            "p_value": p_value,
            "std_err": std_err } })
    return ret

In [9]:
for did in list(registry.queryDimensionRecords("detector",instrument='LSSTCam'))[:189]:
    print(did.id, did.full_name)
    path = f'/home/y/youtsumi/notebooks/Run7/gaintemp/{did.full_name}.pickle'
    print(path)
    if os.path.exists(path):
        continue

    ret={ did.full_name: getcoef(did.id) }
    with open(path,'wb') as f:
        pickle.dump(ret,f)
    
#getcoef('')

0 R01_S00
/home/y/youtsumi/notebooks/Run7/gaintemp/R01_S00.pickle
['u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35/20241025T082704Z', 'u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35'] 0


100%|██████████| 555/555 [00:35<00:00, 15.74it/s]


1 R01_S01
/home/y/youtsumi/notebooks/Run7/gaintemp/R01_S01.pickle
['u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35/20241025T082704Z', 'u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35'] 1


100%|██████████| 555/555 [00:36<00:00, 15.27it/s]


2 R01_S02
/home/y/youtsumi/notebooks/Run7/gaintemp/R01_S02.pickle
['u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35/20241025T082704Z', 'u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35'] 2


...

100%|██████████| 555/555 [00:34<00:00, 16.00it/s]


187 R43_S21
/home/y/youtsumi/notebooks/Run7/gaintemp/R43_S21.pickle
['u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35/20241025T082704Z', 'u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35'] 187


100%|██████████| 555/555 [00:34<00:00, 16.21it/s]


188 R43_S22
/home/y/youtsumi/notebooks/Run7/gaintemp/R43_S22.pickle
['u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35/20241025T082704Z', 'u/lsstccs/eo_flat_gain_stability_E1496_w_2024_35'] 188


100%|██████████| 555/555 [00:34<00:00, 16.22it/s]


In [18]:
pylab.hist([ ret['R01_S01'][x]['y0'] for x in ret['R01_S01'] ])

KeyError: 'R01_S01'

In [8]:
pylab.hist([ ret['R01_S01'][x]['slope'] for x in ret['R01_S01'] ])

KeyError: 'R01_S01'

In [9]:
import pickle
with open('gaintemp.pickle','wb') as f:
    pickle.dump(ret,f)